# ML Pipeline Preparation
Follow the instructions below to help you create your ML pipeline.
### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [20]:
# import libraries
import pandas as pd
import nltk
nltk.download(['punkt', 'wordnet','stopwords'])
import re
import numpy as np
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sqlalchemy import create_engine
from sklearn.multioutput import MultiOutputClassifier
from nltk.corpus import stopwords
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [21]:
# load data from database
engine = create_engine('sqlite:///ETLProject.db')
df = pd.read_sql("dataframe",engine)
# drop any none rows
df = df.dropna()
X = df["message"]
Y = df[['related', 'request', 'offer', 'aid_related', 'medical_help', 'medical_products', 'search_and_rescue', 'security', 'military', 'child_alone', 'water', 'food', 'shelter', 'clothing', 'money', 'missing_people', 'refugees', 'death', 'other_aid', 'infrastructure_related', 'transport', 'buildings', 'electricity', 'tools', 'hospitals', 'shops', 'aid_centers', 'other_infrastructure', 'weather_related', 'floods', 'storm', 'fire', 'earthquake', 'cold', 'other_weather', 'direct_report']]

In [22]:
X

0        Weather update - a cold front from Cuba that c...
1                  Is the Hurricane over or is it not over
2                          Looking for someone but no name
3        UN reports Leogane 80-90 destroyed. Only Hospi...
4        says: west side of Haiti, rest of the country ...
5                   Information about the National Palace-
6                           Storm at sacred heart of jesus
7        Please, we need tents and water. We are in Sil...
8          I would like to receive the messages, thank you
9        I am in Croix-des-Bouquets. We have health iss...
10       There's nothing to eat and water, we starving ...
11       I am in Petionville. I need more information r...
12       I am in Thomassin number 32, in the area named...
13       Let's do it together, need food in Delma 75, i...
14       More information on the 4636 number in order f...
15       A Comitee in Delmas 19, Rue ( street ) Janvier...
16       We need food and water in Klecin 12. We are dy.

In [23]:
Y

,related,request,offer,aid_related,medical_help,medical_products,search_and_rescue,security,military,child_alone,...,aid_centers,other_infrastructure,weather_related,floods,storm,fire,earthquake,cold,other_weather,direct_report
0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
2,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,1.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
7,1.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
8,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
9,1.0,1.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0


### 2. Write a tokenization function to process your text data

In [24]:
def tokenize(text):
    #tokenize the sentence
    tokens = word_tokenize(text)
    
    #normalize and lemmatize words
    stop_words = stopwords.words("english")
    lemmatizer = WordNetLemmatizer()
    
    cleaned_tokens = [lemmatizer.lemmatize(word) for word in tokens if word not in stop_words]
    return cleaned_tokens

### 3. Build a machine learning pipeline
This machine pipeline should take in the `message` column as input and output classification results on the other 36 categories in the dataset. You may find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

In [25]:
pipeline = Pipeline([
    ("vect",CountVectorizer(tokenizer = tokenize)),
    ("tfidf",TfidfTransformer()),
    ("pred",MultiOutputClassifier(RandomForestClassifier()))
])

### 4. Train pipeline
- Split data into train and test sets
- Train pipeline

In [26]:
X_train, X_test, y_train, y_test = train_test_split(X, Y)
#train pipeline
pipeline.fit(X_train,y_train)

Pipeline(memory=None,
     steps=[('vect', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
        strip...oob_score=False, random_state=None, verbose=0,
            warm_start=False),
           n_jobs=1))])

### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

In [27]:
#make prediciton
y_pred = pd.DataFrame(pipeline.predict(X_test), columns = Y.columns)

#generate report for each column
#total f1-score
total = 0
for column in Y.columns.values:
    
    report = classification_report(y_test[column], y_pred[column])
    f1_score = report.split()[-2]
    total += float(f1_score)
    print(column)
    print(report)

#print out total average accuracy
average_accuracy = total / len(Y.columns)
print("The total average accuracy is: ",average_accuracy)

related
             precision    recall  f1-score   support

        0.0       0.38      0.24      0.29       855
        1.0       0.67      0.81      0.73      1667
        2.0       0.50      0.03      0.06        34

avg / total       0.57      0.61      0.57      2556

request
             precision    recall  f1-score   support

        0.0       0.64      0.87      0.74      1646
        1.0       0.36      0.13      0.19       910

avg / total       0.54      0.61      0.54      2556

offer
             precision    recall  f1-score   support

        0.0       1.00      1.00      1.00      2553
        1.0       0.00      0.00      0.00         3

avg / total       1.00      1.00      1.00      2556

aid_related
             precision    recall  f1-score   support

        0.0       0.62      0.83      0.71      1568
        1.0       0.42      0.19      0.26       988

avg / total       0.54      0.58      0.54      2556

medical_help
             precision    recall  f1-sco

/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


### 6. Improve your model
Use grid search to find better parameters. 

In [28]:
parameters = parameters = {
        'pred__estimator__n_estimators': [50, 100, 200],
    }

cv = GridSearchCV(pipeline, param_grid=parameters)
cv.fit(X_train,y_train)

GridSearchCV(cv=None, error_score='raise',
       estimator=Pipeline(memory=None,
     steps=[('vect', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
        strip...oob_score=False, random_state=None, verbose=0,
            warm_start=False),
           n_jobs=1))]),
       fit_params=None, iid=True, n_jobs=1,
       param_grid={'pred__estimator__n_estimators': [50, 100, 200]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring=None, verbose=0)

### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

In [29]:
#make prediciton
new_y_pred = pd.DataFrame(cv.predict(X_test), columns = Y.columns)

In [30]:
#generate report for each column
#total f1-score
total = 0
for column in Y.columns.values:
    
    report = classification_report(y_test[column], new_y_pred[column])
    f1_score = report.split()[-2]
    total += float(f1_score)
    print(column)
    print(report)

#print out total average accuracy
average_accuracy = total / len(Y.columns)
print("The total average accuracy is: ",average_accuracy)

related
             precision    recall  f1-score   support

        0.0       0.39      0.08      0.13       855
        1.0       0.66      0.93      0.77      1667
        2.0       0.25      0.03      0.05        34

avg / total       0.56      0.64      0.55      2556

request
             precision    recall  f1-score   support

        0.0       0.65      0.90      0.75      1646
        1.0       0.39      0.11      0.18       910

avg / total       0.56      0.62      0.55      2556

offer
             precision    recall  f1-score   support

        0.0       1.00      1.00      1.00      2553
        1.0       0.00      0.00      0.00         3

avg / total       1.00      1.00      1.00      2556

aid_related
             precision    recall  f1-score   support

        0.0       0.63      0.86      0.73      1568
        1.0       0.46      0.19      0.27       988

avg / total       0.56      0.60      0.55      2556

medical_help
             precision    recall  f1-sco

/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


### 8. Try improving your model further. Here are a few ideas:
* try other machine learning algorithms
* add other features besides the TF-IDF

In [32]:
#I IMPROVED THE MODEL ON MY LOCAL TRAIN_CLASSIFIER.PY SINCE THIS WORKSPACE DID NOT HAVE XGBOOST

### 9. Export your model as a pickle file

In [33]:
#EXPORTED MODEL AS PICKLE FIL ON TRAIN_CLASSIFIER.PY

### 10. Use this notebook to complete `train.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.